<a href="https://colab.research.google.com/github/Evans-tats/machine-learning/blob/main/RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install swig cmake
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 0s (4,029 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 123621 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubu

In [2]:
#this cell will install virtual screen libraries and run a virtual screen for us
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,449 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,602 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,592 kB]
Hit:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:14 https://ppa.launchpadc

In [ ]:
#this is to reset our runtime to make sure, making our virtual screeen to run
import os
os.kill(os.getpid(), 9)

In [2]:
#virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400,900))
virtual_display.start()


In [3]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

##SETTING UP GYMNASIUM

In [13]:
import gymnasium as gym
#we create an environment
env = gym.make("LunarLander-v2")

#then we reset this environment
observation, info = env.reset()

for _ in range(20):
  # take random action
  action = env.action_space.sample()
  print("action taken: ", action)
  # do this action in the environment
  observation, reward, terminated, truncated, info = env.step(action)

  # if the game is terminated or truncated
  if terminated or truncated:
    #reset environment
    print("environment is reset")
    observation, info = env.reset()
env.close()

action taken:  3
action taken:  3
action taken:  0
action taken:  3
action taken:  1
action taken:  2
action taken:  3
action taken:  1
action taken:  3
action taken:  3
action taken:  1
action taken:  1
action taken:  1
action taken:  2
action taken:  1
action taken:  1
action taken:  2
action taken:  0
action taken:  1
action taken:  0


we create a vectorised environment(a method of stacking multiple independent environments into a single environment) of 16 environments, this way, we'll have ,ore diverse experiences during training

In [12]:
env = make_vec_env('LunarLander-v2', n_envs=16)

##CREATE MODEL


In [16]:
# TODO: Define a PPO MlpPolicy architecture
# We use MultiLayerPerceptron (MLPPolicy) because the input is a vector,
# if we had frames as input we would use CnnPolicy
model = PPO('MlpPolicy', env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [17]:
# We added some parameters to accelerate the training
model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [19]:
# TODO: Specify file name for model and save the model to file
model_name = "ppo-LunarLander-v2"

In [22]:
model.learn(total_timesteps=1000000)

Streaming output truncated to the last 5000 lines.
|    value_loss           | 128           |
-------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 840          |
|    ep_rew_mean          | 124          |
| time/                   |              |
|    fps                  | 543          |
|    iterations           | 740          |
|    time_elapsed         | 1392         |
|    total_timesteps      | 757760       |
| train/                  |              |
|    approx_kl            | 0.0016361865 |
|    clip_fraction        | 0.000488     |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.725       |
|    explained_variance   | 0.494        |
|    learning_rate        | 0.0003       |
|    loss                 | 2.64         |
|    n_updates            | 2960         |
|    policy_gradient_loss | -0.000631    |
|    value_loss           | 17.2         |
-

##EVALUATE AGENT

In [23]:
eval_env = Monitor(gym.make("LunarLander-v2", render_mode='rgb_array'))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=240.97 +/- 21.443865840055754


In [24]:
notebook_login()
!git config --global credential.helper store

In [25]:
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

## TODO: Define a repo_id
## repo_id is the id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
repo_id = "Tatsss/ppo-LunarLander-v2"

# TODO: Define the name of the environment
env_id = "LunarLander-v2"

# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])


# TODO: Define the model architecture we used
model_architecture = "PPO"

## TODO: Define the commit message
commit_message = "Upload PPO LunarLander-v2 trained agent"

# method save, evaluate, generate a model card and record a replay video of your agent before pushing the repo to the hub
package_to_hub(model=model, # Our trained model
               model_name=model_name, # The name of our trained model
               model_architecture=model_architecture, # The model architecture we used: in our case PPO
               env_id=env_id, # Name of the environment
               eval_env=eval_env, # Evaluation Environment
               repo_id=repo_id, # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
               commit_message=commit_message)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Saving video to /tmp/tmpf5su2k0n/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpf5su2k0n/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpf5su2k0n/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpf5su2k0n/-step-0-to-step-1000.mp4
ℹ Pushing repo Tatsss/ppo-LunarLander-v2 to the Hugging Face Hub


...

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

policy.pth:   0%|          | 0.00/43.8k [00:00<?, ?B/s]

pytorch_variables.pth:   0%|          | 0.00/864 [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/Tatsss/ppo-LunarLander-v2/tree/main/


CommitInfo(commit_url='https://huggingface.co/Tatsss/ppo-LunarLander-v2/commit/5d45676d8c02df8c592c0ba5d6c3d6a32cbace64', commit_message='Upload PPO LunarLander-v2 trained agent', commit_description='', oid='5d45676d8c02df8c592c0ba5d6c3d6a32cbace64', pr_url=None, pr_revision=None, pr_num=None)